In [1]:
# importando bibliotecas auxiliares
import math, random
import numpy as np

In [2]:
# lendo os arquivos de entrada
def read_initial_weights(path_file):
    '''função responsavel em ler o arquivo de pesos inicias'''
    return open(path_file,'r+').readlines()

def read_dataset(path_file):
    '''função responsável em ler o arquivo de dataset'''
    dataset = list()
    file = open(path_file, 'r+')
    for lines in file:
        values_line = list()
        dataset.append([lines.rstrip()])
    return dataset
    
def read_def_network(path_file):
    '''função responsável em ler o arquivo de definição da rede neural'''
    file = open(path_file, "r+")
    
    network = list()
    for line in file:
        network.append(line.rstrip())
    return network

initial_weights = read_initial_weights('initial_weights.txt')
print('initial_weights:\n', initial_weights)

dataset = read_dataset("dataset.txt")
print('dataset:\n', dataset)

def_network = read_def_network('network.txt')
print('def_network:\n', def_network)

initial_weights:
 ['0.40000, 0.10000; 0.30000, 0.20000\n', '0.70000, 0.50000, 0.60000']
dataset:
 [['0.13000; 0.90000'], ['0.42000; 0.23000']]
def_network:
 ['0.250', '1', '2', '1']


#### Rede neural composta por um número ajustável de neurônios e camadas e treinada via backpropagation

In [3]:
def create_network(def_network, initial_weights):
    '''função responsável em criar a rede neural a partir dos arquivos de definição'''
    '''def_network definição da estrutura da rede'''
    '''initial_weights pesos inicias da rede'''
    network = list()
    
    # trabalhando sobre o arquivo que define a rede
    regulation = def_network[0]
    def_network.pop(0) # remove a regulação já armazenada
    
    input_length = def_network[0]
    def_network.pop(0)
    
    output_length = def_network[len(def_network)-1]
    def_network.pop(len(def_network)-1)
    
    # imprimindo informações capturadas da definição da rede
    print('[INFO] Regulação: ', regulation)
    print('[INFO] Quantidade de entradas: ', input_length)
    print('[INFO] Quantidade de saídas: ', output_length)
    # imprimindo informações das camadas ocultas
    print('[INFO] Número de neurônios em cada camada oculta: ', def_network, '\n')
    
    # trabalhando sobre o arquivo de quefine os pesos
    for weights in initial_weights:
        layers = weights.rstrip().split('; ')
        network_layer = list()
        for neuron in layers:
            if ', ' in neuron:
                network_layer.append({'weights': neuron.rstrip().split(', ')})
            else:
                network_layer.append(neuron)
        network.append(network_layer)
    print('[INFO] Rede neural criada: ', network, '\n')
    return network

def dot(v, w):
    print(w)    return sum(float(v_i) * float(w_i) for v_i, w_i in zip(v, w))

def sigmoid(activation):
    '''função responsável por calcular a aproximação da função'''
    return 1 / (1 + math.exp(-activation))

def forward_propagate(network, row):
    ''''''
    inputs = row
    
    index = 0
    for layer in network:
        new_inputs = list()
        if index != len(network)-1:
            new_inputs.append(1.00000) # adicionando a entrada do vies
        for neuron in layer:
            activation = dot(neuron['weights'], inputs)
            neuron['output'] = sigmoid(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
        index = index + 1
    return inputs


# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (float(neuron['weights'][j]) * neuron['delta'])
                    errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
            print('errors', errors)
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']
            
# construindo a rede neural
network = create_network(def_network, initial_weights)  # TODO: mudar para o arquivo def a rede
row = [1.00000, 0.13000] # adicionando a entrada do vies
output = forward_propagate(network, row)
print('[INFO] valor de saída (fx): ', output, '\n')

backward_propagate_error(network, [0.90000])

# visualizando a rede com os pesos calculados
for layer in network:
    for neuron in layer:
        print(neuron)
    print('\n')

[INFO] Regulação:  0.250
[INFO] Quantidade de entradas:  1
[INFO] Quantidade de saídas:  1
[INFO] Número de neurônios em cada camada oculta:  ['2'] 

[INFO] Rede neural criada:  [[{'weights': ['0.40000', '0.10000']}, {'weights': ['0.30000', '0.20000']}], [{'weights': ['0.70000', '0.50000', '0.60000']}]] 

w [1.0, 0.13]
w [1.0, 0.13]
w [1.0, 0.6018070039467833, 0.5807857981259906]
[INFO] valor de saída (fx):  [0.7940274264318581] 

errors [0.10597257356814194]
{'weights': ['0.40000', '0.10000'], 'output': 0.6018070039467833, 'delta': 0.0029072827742848578}
{'weights': ['0.30000', '0.20000'], 'output': 0.5807857981259906, 'delta': 0.0021098926528837765}


{'weights': ['0.70000', '0.50000', '0.60000'], 'output': 0.7940274264318581, 'delta': 0.017331588951040165}




#### Funcionalidade que permita, via linha de comando, informar a sua implementação a estrutura de uma rede de teste (i.e., estrutura de camadas/neurônios, pesos iniciais, e fator de regularização), e um conjunto de treinamento, e que retorne o gradiente calculado para cada peso

In [4]:
0.60874   0.59484
0.79597

SyntaxError: invalid syntax (<ipython-input-4-c98ca529052f>, line 1)

#### Funcionalidade que permita, via linha de comando, efetuar a verificação numérica do gradiente, a fim de checar a corretude da implementação de cada grupo

#### Um mecanismo para normalização das features/dados de treinamentos

#### Mecanismo para uso de regularização